In [3]:
import sys

sys.path.append('../')
from auton_survival import datasets
outcomes, features = datasets.load_support()

D:\anaconda3\envs\FLCausalMixture\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
from auton_survival.preprocessing import Preprocessor

cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp',
             'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph',
             'glucose', 'bun', 'urine', 'adlp', 'adls']

# Data should be processed in a fold-independent manner when performing cross-validation.
# For simplicity in this demo, we process the dataset in a non-independent manner.
preprocessor = Preprocessor(cat_feat_strat='ignore', num_feat_strat= 'mean')
x = preprocessor.fit_transform(features, cat_feats=cat_feats, num_feats=num_feats,
                                one_hot=True, fill_value=-1)

In [5]:
import numpy as np
times = np.quantile(outcomes.time[outcomes.event==1], [0.25, 0.5, 0.75]).tolist()

In [6]:
from auton_survival.experiments import SurvivalRegressionCV

param_grid = {'k' : [3],
              'distribution' : ['Weibull'],
              'learning_rate' : [1e-4, 1e-3],
              'layers' : [[100]]}

experiment = SurvivalRegressionCV(model='dsm', num_folds=3, hyperparam_grid=param_grid, random_seed=0)
model = experiment.fit(x, outcomes, times, metric='brs')

At hyper-param {'distribution': 'Weibull', 'k': 3, 'layers': [100], 'learning_rate': 0.0001}
At fold: 0


100%|██████████| 50/50 [00:06<00:00,  7.97it/s]


At fold: 1


100%|██████████| 50/50 [00:06<00:00,  7.99it/s]


At fold: 2


...

100%|██████████| 50/50 [00:06<00:00,  7.79it/s]


At fold: 1


100%|██████████| 50/50 [00:06<00:00,  7.60it/s]


At fold: 2


100%|██████████| 50/50 [00:09<00:00,  5.13it/s]


In [7]:
print(experiment.folds)
model

[2 2 0 ... 0 0 0]


In [8]:
out_risk = model.predict_risk(x, times)
out_survival = model.predict_survival(x, times)

In [9]:
from auton_survival.metrics import survival_regression_metric

for fold in set(experiment.folds):
    print(survival_regression_metric('brs', outcomes[experiment.folds==fold],
                                     out_survival[experiment.folds==fold],
                                     times=times))

[0.12917649 0.19436242 0.20631976]
[0.12691664 0.19280962 0.20635106]
[0.12159038 0.19158577 0.20871653]


D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \
D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \
D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


In [10]:
from auton_survival.metrics import survival_regression_metric

for fold in set(experiment.folds):
    print(survival_regression_metric('ctd', outcomes[experiment.folds==fold],
                                     out_survival[experiment.folds==fold],
                                     times=times))

D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


[0.765472457063747, 0.7223859858263633, 0.688765174086125]


D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


[0.781271531344058, 0.7275326444323827, 0.688923316104416]


D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


[0.7612520620726958, 0.7181912109861889, 0.6816684343816966]


In [11]:
for fold in set(experiment.folds):
    for time in times:
        print(time)

14.0
58.0
252.0
14.0
58.0
252.0
14.0
58.0
252.0
